In [4]:
%run load_modules.ipynb
kB = 1.381e-23 # Boltzmann Constant
T = 300 # Temperature
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 10 # Post Load Amplifier GainkB = 1.381e-23 # Boltzmann Constant
T = 300 # Temperature
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 100 # Post Load Amplifier Gain
bandwidth0 = 500 # Unfilitered Bandwidth
Pave0 =30e-6
bandwidth = 20 # The 3 dB bandwidth of the receiver

sigmaT = np.sqrt(4*kB*T/RL*bandwidth0*1e9)*G
Pave = 30e-6; sigma0 = sigmaT; sigma1 = sigmaT # Divide Pave by 1000 to give voltage in mV
Q2 = (2*Pave0*R*G/(sigma0*(bandwidth/bandwidth0)**0.5+sigma1*(bandwidth/bandwidth0)**0.5)) # The Q-Factor that gives BER
BER2 = 0.5*special.erfc(Q2/np.sqrt(2)) # Bit Error Rate

v12 = np.random.normal(0, sigma0*RL, 400)
v22 = np.random.normal(Pave0*2*R*RL*G, sigma1*RL, 400)
v32 = np.random.normal(0, sigma0*RL, 400)
v2 = np.concatenate([v12,v22,v32])
t=np.linspace(0,1200,1200) # time in picoseconds
zeros=np.zeros(1200)
Threshold2=zeros+Pave*R*RL*G

sig_fft= np.fft.rfft(v2)
sig_fft_norm = np.absolute(sig_fft)/np.amax(np.absolute(sig_fft))
fft_freq = np.linspace(0,600/1.200,601)
filter=np.heaviside(bandwidth0-fft_freq, 0*fft_freq)

p2 = figure(width=400, height=400,y_axis_type="linear", background_fill_color="lightgray")
p2.outline_line_width = 1; p2.outline_line_color = "black"; p2.min_border_top = 10
lfft = p2.line(fft_freq,sig_fft_norm,line_color='#2222aa', legend_label = 'Frequency Components')
p2.legend.background_fill_alpha = 0.5
p2.legend.location='top_right'
p2.x_range = Range1d(0, 50); p2.y_range = Range1d(0.01, 1.05)
lf = p2.line(fft_freq,filter, line_color='red', line_width=2, legend_label = 'Filter Transmission')
p2.xaxis.axis_label="Frequency (GHz)"; p2.xaxis.major_label_text_font_size = "12pt"
p2.xaxis.axis_label_text_font_style = "normal"; p2.xaxis.axis_label_text_font_size = "12pt"
p2.yaxis.axis_label="Normalized Amplitude"; p2.yaxis.major_label_text_font_size = "12pt"
p2.yaxis.axis_label_text_font_style = "normal"; p2.yaxis.axis_label_text_font_size = "12pt"

sig_fft_f = sig_fft*filter
inv_sig_fft=np.fft.irfft(sig_fft_f)

p3 = figure(width=400, height=400, background_fill_color="lightgray",
            title="Bandwidth Limited Signal")
p3.title.text_font_style='normal'; p3.title.text_font_size='12pt'
p3.outline_line_width = 1; p3.outline_line_color = "black"
p3.x_range = Range1d(0, 1200);  p3.y_range= Range1d(-0.1,0.4)
lbl = p3.line(t,inv_sig_fft,line_color='#2222aa', line_alpha=0.4)
lb2 = p3.line(t,inv_sig_fft,line_color='#2222aa')
lt2 = p3.line(t,Threshold2,line_dash='dashed',line_color='red')
p3.xaxis.axis_label="Time (psec)"; p3.xaxis.major_label_text_font_size = "12pt"
p3.xaxis.axis_label_text_font_style = "normal"; p3.xaxis.axis_label_text_font_size = "12pt"
p3.yaxis.axis_label="Amplitude"; p3.yaxis.major_label_text_font_size = "12pt"
p3.yaxis.axis_label_text_font_style = "normal"; p3.yaxis.axis_label_text_font_size = "12pt"

p3.add_layout(Label(x=200, y=300, text='0', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))
p3.add_layout(Label(x=600, y=300, text='1', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))
p3.add_layout(Label(x=1000, y=300, text='0', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))

label4=Label(x=20, y=Pave*R*RL*G, text='V Threshold')
label5=Label(x=20,  y=0, y_units='screen', text='BER='+'{:.2e}'.format(BER2),
                                             text_color='darkblue', background_fill_color='lightgray')
p3.add_layout(label4)
p3.add_layout(label5)

"""
style = {'description_width': 'initial'}
bandwidth_slider=FloatSlider(min=1, max=51, step=1, value = 20.0, description='Bandwidth (GHz)',
                             continuous_update=True, style=style)
Pave_slider2=FloatSlider(min=0, max=60, step=1, value = 30, description='Pave (\u03BCW)', continuous_update=True)
jitter_slider=FloatSlider(min=-140, max=140, step=20, value = 0, description='Jitter (psec)',
                             continuous_update=True, style=style)
"""

bandwidth_slider=Slider(start=1, end=51, step=1, value = 20.0, title='Bandwidth (GHz)')
Pave_slider2=Slider(start=0, end=60, step=1, value = 30, title='Pave (\u03BCW)')
jitter_slider=Slider(start=-140, end=140, step=20, value = 0, title='Jitter (psec)')

def replot2(attrname, old, new):
# def replot2(bandwidth, jitter, Pave):
    Pave=Pave_slider2.value*1e-6
    v22_local=v22
    v22_local = v22_local - Pave0*2*R*RL*G +Pave*2*R*RL*G
    v2 = np.concatenate([v12,v22_local,v32])
    sig_fft= np.fft.rfft(v2)
    filter=np.heaviside(bandwidth_slider.value-fft_freq,1)
    lf.data_source.data['y']=filter
    sig_fft_f = sig_fft*filter
    inv_sig_fft=np.fft.irfft(sig_fft_f)
    lbl.data_source.data['y']=inv_sig_fft
    lb2.data_source.data['x']=t-jitter_slider.value
    lb2.data_source.data['y']=inv_sig_fft
    lt2.data_source.data['y']=zeros+np.average(inv_sig_fft[:2000])
    label4.y=np.average(inv_sig_fft[:2000])
    Q2 = (2*Pave*R*G/(sigma0*(bandwidth/bandwidth0)**0.5+sigma1*(bandwidth/bandwidth0)**0.5)) # The Q-Factor that gives BER
    BER2 = 0.5*special.erfc(Q2/np.sqrt(2)) # Bit Error Rate'
    label5.text='BER='+'{:.2e}'.format(BER2)
#    push_notebook(handle=fig_handle2)
# fig_handle2 = show(row(p2,p3), notebook_handle=True)
# interact(replot2, bandwidth=bandwidth_slider, jitter=jitter_slider, Pave=Pave_slider2);

bandwidth_slider.on_change('value',replot2)
Pave_slider2.on_change('value',replot2)
jitter_slider.on_change('value',replot2)                    
bokeh_app2 = pn.pane.Bokeh(column(row(p2,p3),bandwidth_slider,Pave_slider2,jitter_slider))
replot2(None,None,None)
# bokeh_app2

Loading BokehJS ...

Bokeh(Column)